Let us try to implement a fast transform as a product of sparse matrices and check whether this implementation is actually faster than the naive dense matrix/vector product.

**Warning:** versioning a notebook is not trivial and you should be very careful since every committed version cannot be deleted!

A notebook file contains both:

* the source code (cells): it is stored efficiently in text format, and does not change when runnning the notebook.
* and, when computed, the results: they are stored in binary format, not efficiently, changing every time the notebook is run, even if no change is made in the source code.

So please be very careful:

* never commit a notebook that has been run and where results are displayed;
* before committing changes in a notebook, just execute `Restard and Clear Output` in the `Kernel` menu: this will delete all the binary output and only the source code will be committed; another option is to run `git checkout my_notebook.ipynb` in order to revert to the commited version and to ignore the delete the current version (assuming that there is no change you want to commit).
* if you are used to committing all the files that have been changed (e.g. if you use a graphical interface for git or if you use option `git commit -a`), then you may commit binary contents of a notebook by mistake; in this case, you may copy the notebook and run the copied non-versionned file (not using the committed version);
* if you have any doubt or question, ask before any commit.

Note that we may find a better solution ([like this one](https://stackoverflow.com/questions/18734739/using-ipython-notebooks-under-version-control/20844506#20844506)) but this is something discussed several times with Florent and Denis and that is not trivial.


In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix

In [ ]:
from time import perf_counter

In [ ]:
from scipy.linalg import hadamard as sp_hadamard_matrix

The Hadamard transform of a vector $v$ in dimension $n$ can be computed as the product $ F^{\log_2 n} v$ where $F$ is a sparse $n \times n$ matrix. The following function builds the $F$ matrix.

In [ ]:
def build_hadamard_factor(n):
    """
    Build the sparse matrix used to compute the fast Hadamard transform.
    
    Parameters
    ----------
    n : int
        Size of the transform
    
    Returns
    -------
    csr_matrix
        Sparse n-by-n matrix
    """
    F = np.zeros((n, n), dtype=int)
    F[range(0, n, 2), range(0, n//2)] = 1
    F[range(1, n, 2), range(0, n//2)] = 1
    F[range(0, n, 2), range(n//2, n)] = 1
    F[range(1, n, 2), range(n//2, n)] = -1
    return csr_matrix(F)

In [ ]:
def build_hadamard_matrix(n):
    """
    Build the dense matrix used to compute the (slow) Hadamard transform.
    
    Parameters
    ----------
    n : int
        Size of the transform
    
    Returns
    -------
    nd-array
        Hadamard n-by-n matrix
    """
    F = build_hadamard_factor(n).toarray()
    H = np.linalg.matrix_power(F, int(np.log2(n)))
    return H

Let us test the function and check that our implementation of Hadamard matrices gives the correct result (i.e., the same result as function `scipy.linalg.hadamard`).

In [ ]:
plt.imshow(build_hadamard_matrix(32))
print(np.all(sp_hadamard_matrix(32) == build_hadamard_matrix(32)))

Here is the main function that implements the fast Hadamard transform.

In [ ]:
def apply_hadamard(v, F=None):
    """
    Apply fast Hadamard transform using the product of sparse matrices.
    
    The Hadamard transform of a vector $v$ in dimension $n$ can be computed as the product $ F^{\log_2 n} v$
    where $F$ is a sparse $n \times n$ matrix
    
    Parameters
    ----------
    v : nd-array [n]
        Vector to be transformed
    
    F : nd-array [n, n], optional
        Sparse factor to compute the Hadamard transform.
        If `None`, the matrix is computed using function `build_hadamard_factor`.
    
    Returns
    -------
    nd-array [n]
        Vector resulting form the Hadamard transform of v.
    """
    n = v.size
    if F is None:
        F = build_hadamard_factor(n)
    for i in range(int(np.log2(n))):
        v = F.dot(v)
    return v

Let us check that the fast transform give the same result as the matrix-vector product.

In [ ]:
n = 32
v = np.random.randn(n)
np.max(np.abs(apply_hadamard(v) - build_hadamard_matrix(n) @ v))

Now, we can measure and compare the running times of the fast transform vs. the product between the dense Hadamard matrix and a data vector.

In [ ]:
t_sparse_matrix = []
t_full_matrix = []
res = []
p_values = np.arange(16)
n_trials = 50
for n in 2**p_values:
    print(n)
    v = np.random.randn(n)
    
    F = build_hadamard_factor(n)
    cum_time = 0
    for _ in range(n_trials):
        t0 = perf_counter()
        v_sparse = apply_hadamard(v, F)
        cum_time += perf_counter() - t0
    t_sparse_matrix.append(cum_time / n_trials)
    
    # Beyond some size limit, skip the computation of the slow method
    if n <= 1024:
        H = build_hadamard_matrix(n)
        cum_time = 0
        for _ in range(n_trials):
            t0 = perf_counter()
            v_full = H @ v
            cum_time += perf_counter() - t0
        t_full_matrix.append(cum_time / n_trials)
        res.append(np.max(np.abs(v_full - v_sparse)))
    else:
        t_full_matrix.append(np.nan)
        res.append(np.nan)


In [ ]:
plt.semilogy(2**p_values, t_full_matrix, label='full')
plt.semilogy(2**p_values, t_sparse_matrix, label='sparse')
plt.legend()
plt.grid()
plt.ylabel('Average running time ({} trials)'.format(n_trials))
plt.xlabel('Transform length')

plt.figure()
plt.plot(res)
plt.ylabel('residual error ($\|.\|_\infty$)')
plt.xlabel('Transform length')
